# Predicting the Past, United States Extension

This is an extension of Predicting the Past, United States. 
The extension includes comparisons between Cyclus output and CURIE data at year 2020 for: 
 - Total spent fuel mass 
 - Mass of major isotopes in spent fuel 

The subsequent sections that set up Cyclus input files from `vision_recipes.xls` recipes and `reactors_pris_2016.csv` deployment data are taken from the Predicting the Past, United States ipynb. 